This notebook uses Random Forest model to predict HPI trends based on economic and environmental factors. I create models both for each of the economic regions and the entire U.S. In order to determine the relative importance of the environmental factors, I compare $R^2$ from a base model, which uses only economic predictors, and a total model, which uses both economic and environmental predictors. For the total model, I also use "feature extractor" of the Random Forest models to obtain the top three most important features based on model coefficients. Results obtained here are plotted in the "DataPlotting_RandomForest.ipynb" notebook, and all models are pickled to be used for later.

In [4]:
import pandas as pd
import numpy as np

from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
import pickle

In [2]:
# Load in the dataframe with economic and environmental data tabulated at 3-digit zipcode level

master_df = pd.read_pickle('master_econ_env_data_zcta.pkl')
master_df.head(5)

,Zipcode,AnnualTrend_2000_2021,AnnualTrend_2000_2008,AnnualTrend_2015_2021,min_year,max_year,INTPTLAT,INTPTLONG,ALAND_SQMI,AWATER_SQMI,...,county,state,region,DP04_0046E,DP04_0047E,Owner_Renter_ratio,Median_income,County,GDP_trend,GDP_trend_norm
0,010,0.031272,0.086983,0.053891,2000.0,2021.0,42.269354,-72.567846,1276.131,38.492,...,Hampden,Massachusetts,New England,122666,59881,2.048496,72810.035088,Hampden,-92137.8,-0.004533
1,011,0.031176,0.080476,0.075606,2000.0,2021.0,42.106624,-72.548348,40.946,1.716,...,Hampden,Massachusetts,New England,31343,30622,1.023545,48369.666667,Hampden,-92137.8,-0.004533
2,012,0.031742,0.091175,0.051823,2000.0,2021.0,42.341925,-73.227852,902.053,16.827,...,Berkshire,Massachusetts,New England,37865,16825,2.250520,69854.100000,Berkshire,-133415.8,-0.020369
3,013,0.031539,0.086385,0.057243,2000.0,2021.0,42.593027,-72.568520,786.429,27.536,...,Franklin,Massachusetts,New England,24277,10953,2.216470,64549.137931,Franklin,1203.2,0.000441
4,014,0.022781,0.055906,0.076745,2000.0,2021.0,42.589492,-71.794956,474.266,14.954,...,Worcester,Massachusetts,New England,57213,25558,2.238555,93780.047619,Worcester,180537.5,0.004647


In [8]:
# Load in dictionary with states divided into regions
state_region_dict = pickle.load(open("state_region_dict.pkl", "rb"))
region_list = list(state_region_dict.keys())

In [6]:
# Now make a Random Forest Regressor pipeline
# ...will be using 2nd order polynomial features with interactions only (i.e., only xy, not x^2 or y^2 terms)
# in order to make the model more complex/robust

pipe_rf = Pipeline([('scaler',StandardScaler()),
    ('poly',PolynomialFeatures(degree=2,interaction_only=True, include_bias=True)),
    ('rf',RandomForestRegressor(n_estimators=10))])

In [12]:
gs.best_estimator_.named_steps['poly'].get_feature_names_out(X1.columns)

array(['1', 'DP04_0089E', 'Owner_Renter_ratio', 'Median_income',
       'GDP_trend_norm', 'DP04_0089E Owner_Renter_ratio',
       'DP04_0089E Median_income', 'DP04_0089E GDP_trend_norm',
       'Owner_Renter_ratio Median_income',
       'Owner_Renter_ratio GDP_trend_norm',
       'Median_income GDP_trend_norm'], dtype=object)

In [13]:
## Now time to loop over each region to make a region dependent model.
## I will make both base model (using only economic factors) and total model (using both economic and environmental 
##        factors). Also, I will use GridSearchCv to find the min_samples_split for each of the Random Forest models.
## In addition to regional models, I will also make a model using all of the U.S. data.

# make dfs to record R^2 and model coefficients from each Random Forest model
r2_df = pd.DataFrame(columns=['region','r2', 'cat'])
coeff_df = pd.DataFrame(columns=['region','coeff1','coeff2','coeff3'])

j=0
for i in range(len(region_list)): #loop over regions
    df_sliced = master_df.loc[master_df['region']==region_list[i]]
    # true values for HPI trends
    y = df_sliced['HPI_Trend_diff_2008_2018']
    # input features for the base (econ-only) model
    X1 = df_sliced[['DP04_0089E','Owner_Renter_ratio','Median_income','GDP_trend_norm']]
    # input features for the total (econ+env) model
    X2 = df_sliced[['DP04_0089E','Owner_Renter_ratio','Median_income','GDP_trend_norm',
                    'NO2_Monthly_AVG','VegInd_AnnualTrend','VegInd_Monthly_AVG',
                    'LandTemp_AnnualTrend','LandTemp_Monthly_AVG','Precip_AnnualTrend',
                    'Precip_Monthly_AVG']]

    gs = GridSearchCV(pipe_rf,{'rf__min_samples_split':[5,10,15,20,25,30,50]})
    gs.fit(X1,y)                 
    
    r2_df.loc[j,'r2'] = (gs.score(X1,y))
    r2_df.loc[j,'region']=region_list[i]
    r2_df.loc[j,'cat']='Econ only'
    j+=1
            
    gs = GridSearchCV(pipe_rf,{'rf__min_samples_split':[5,10,15,20,25,30,50]})
    gs.fit(X2,y)             
    r2_df.loc[j,'r2'] = (gs.score(X2,y))
    r2_df.loc[j,'region']=region_list[i]
    r2_df.loc[j,'cat']='Total'
    
    #save each regional model
    filename = 'new_RFmodel_'+region_list[i]+'.sav'
    pickle.dump(gs,open(filename,'wb'))

    #Extract top three features by importance
    coef_list = gs.best_estimator_.named_steps['poly'].get_feature_names_out(X2.columns)
    coefs = gs.best_estimator_.named_steps['rf'].feature_importances_
    coeff_ind_max = np.argsort(-abs(coefs))
    coeff_df.loc[i,'region']=region_list[i]
    coeff_df.loc[i,'coeff1'] = coef_list[coeff_ind_max[0]]
    coeff_df.loc[i,'coeff2'] = coef_list[coeff_ind_max[1]]
    coeff_df.loc[i,'coeff3'] = coef_list[coeff_ind_max[2]]
    j+=1

#Repeating the process for the entire U.S.
y = master_df['HPI_Trend_diff_2008_2018']
X1 = master_df[['DP04_0089E','Owner_Renter_ratio','Median_income','GDP_trend_norm']]
X2 = master_df[['DP04_0089E','Owner_Renter_ratio','Median_income','GDP_trend_norm',
                'NO2_Monthly_AVG','VegInd_AnnualTrend','VegInd_Monthly_AVG',
                'LandTemp_AnnualTrend','LandTemp_Monthly_AVG','Precip_AnnualTrend',
                'Precip_Monthly_AVG']]

gs = GridSearchCV(pipe_rf,{'rf__min_samples_split':[5,10,15,20,25,30,50]})
gs.fit(X1,y)             
r2_df.loc[j,'r2'] = (gs.score(X1,y))
r2_df.loc[j,'region']='All U.S.'
r2_df.loc[j,'cat']='Econ only'
j+=1

gs = GridSearchCV(pipe_rf,{'rf__min_samples_split':[5,10,15,20,25,30,50]})
gs.fit(X2,y)             
r2_df.loc[j,'r2'] = (gs.score(X2,y))
r2_df.loc[j,'region']='All U.S.'
r2_df.loc[j,'cat']='Total'
coef_list = gs.best_estimator_.named_steps['poly'].get_feature_names_out(X2.columns)
coefs = gs.best_estimator_.named_steps['rf'].feature_importances_
coeff_ind_max = np.argsort(-abs(coefs))
coeff_df.loc[i+1,'region']='All U.S.'
coeff_df.loc[i+1,'coeff1'] = coef_list[coeff_ind_max[0]]
coeff_df.loc[i+1,'coeff2'] = coef_list[coeff_ind_max[1]]
coeff_df.loc[i+1,'coeff3'] = coef_list[coeff_ind_max[2]]

In [14]:
# Take a look at the three most important features for each region:
coeff_df

,region,coeff1,coeff2,coeff3
0,New England,GDP_trend_norm,DP04_0089E Precip_Monthly_AVG,Median_income GDP_trend_norm
1,Mideast,DP04_0089E,LandTemp_Monthly_AVG,VegInd_Monthly_AVG LandTemp_Monthly_AVG
2,Great Lakes,DP04_0089E,DP04_0089E NO2_Monthly_AVG,LandTemp_AnnualTrend
3,Plains,LandTemp_Monthly_AVG,VegInd_Monthly_AVG LandTemp_Monthly_AVG,VegInd_Monthly_AVG
4,South East,LandTemp_Monthly_AVG Precip_Monthly_AVG,DP04_0089E LandTemp_Monthly_AVG,NO2_Monthly_AVG
5,South West,Median_income,GDP_trend_norm,GDP_trend_norm VegInd_AnnualTrend
6,Far West,DP04_0089E,NO2_Monthly_AVG VegInd_AnnualTrend,NO2_Monthly_AVG
7,Rocky Mountain,GDP_trend_norm,LandTemp_AnnualTrend,NO2_Monthly_AVG
8,All U.S.,DP04_0089E,GDP_trend_norm,DP04_0089E NO2_Monthly_AVG


In [15]:
# Take a look at R^2 of the base and total models fore each region
r2_df

,region,r2,cat
0,New England,0.423716,Econ only
1,New England,0.576254,Total
2,Mideast,0.785339,Econ only
3,Mideast,0.80447,Total
4,Great Lakes,0.818537,Econ only
5,Great Lakes,0.733069,Total
6,Plains,0.372239,Econ only
7,Plains,0.897595,Total
8,South East,0.581327,Econ only
9,South East,0.757488,Total


In [16]:
# Save dataframes for coefficients and R^2 for future use
r2_df.to_pickle('random_forest_r2.pkl')
coeff_df.to_pickle('random_forest_top3_features.pkl')